# Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import trange, tqdm

import torch as tc
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [ ]:
# we are no longer doing the BGR to RGB conversion here
# we'll be using PIL instead of OpenCV to load images
def show_img(im, ax=None, figsize=(8,8), title=None):
    if not ax: _,ax = plt.subplots(1,1,figsize=figsize)
    if len(im.shape)==2: im = np.tile(im[:,:,None], 3)
    ax.imshow(im);
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)
    if title: ax.set_title(title)
    return ax

In [ ]:
def show_imgs(ims, rows=1, figsize=(16,8), title=[None]):
    title = title*len(ims) if len(title) == 1 else title
    _,ax = plt.subplots(rows, len(ims)//rows, figsize=figsize)
    [show_img(im,ax_,title=tit) for im,ax_,tit in zip(ims,ax.flatten(),title)]
    return ax

# Convolutions

## The fully-connected layer problem

Let's assume that we want to build a neural network to classify colour images of relatively high `4096x4096` resolution into one of `100` classes.

In [ ]:
c, h, w = 3, 4096, 4096

In [ ]:
# batch of dummy data (BxCxHxW)
xb = tc.randn((16, c, h, w))
xb[0,0,:5,:5]

In [ ]:
# Logistic Regression network - this is supposed to crash!
net = nn.Sequential(
    nn.Linear(c*h*w, 100),
    nn.LogSoftmax(dim=1)
).cuda()

In [ ]:
out = net(xb.view(xb.shape[0],-1).cuda())

In [ ]:
c*h*w*100*4 / 1024**3

## Hierarchical structure of images

Images are made of elementary building blocks like *blobs* and *edges*. These can be combined to form more advanced shapes like *corners*. These in turn can lead to geometric shapes like *squares* etc. Following this reasoning one can eventually arrive at detailed real-life objects!

An important thing to realise is that an *edge* can (and will) appear anywhere in an image! The fully-connected/linear layer can't take advantage of that because each of its weights is hardwired to a particular input pixel.

## Convolutions explained

- Convolutions walkthrough in [Excel](https://livebournemouthac-my.sharepoint.com/:x:/g/personal/mbudka_bournemouth_ac_uk/EfUMp0FVIaFJl0UboP4xb3gB8LyUPGeeV_yBTnWiBDBp4A?e=ueJMEu)
- Yann Lecun's [paper on convolutions](http://www.max.hi-ho.ne.jp/kindo/sail_files/references/CV-Projects/lecun-89e.pdf) from **1989!!**
- [LeNet-5 from 1998](http://vision.stanford.edu/cs598_spring07/papers/Lecun98.pdf) is essentially the relatively modern architecture of [AlexNet](https://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks.pdf) from 2012

# Simple CNN

In [ ]:
net = nn.Sequential(
    nn.Conv2d(in_channels=3,  out_channels=16, kernel_size=3, stride=1, padding=1), # 1 pixel padding with a 3x3 kernel preserves resolution
    nn.LeakyReLU(inplace=True),
    nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3, stride=1, padding=1), # for 5x5 kernel, padding=2 would preserve resolution
    nn.LeakyReLU(inplace=True),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1),
    nn.LeakyReLU(inplace=True),
    nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, stride=1, padding=1),
    nn.LeakyReLU(inplace=True),
    nn.MaxPool2d(kernel_size=2, stride=2),
).cuda()

In [ ]:
xb = tc.randn((16, 3, 256, 256)).cuda()

In [ ]:
out = net(xb)
out.shape

## Shapes step by step

In [ ]:
xb.shape

In [ ]:
o1 = net[0](xb)
net[0]

In [ ]:
o1.shape

In [ ]:
o2 = net[1](o1)
net[1]

In [ ]:
o2.shape

In [ ]:
o3 = net[2](o2)
net[2]

In [ ]:
o3.shape

In [ ]:
o4 = net[3](o3)
net[3]

In [ ]:
o4.shape

In [ ]:
o5 = net[4](o4)
net[4]

In [ ]:
o5.shape

In [ ]:
o6 = net[5](o5)
net[5]

In [ ]:
o6.shape

In [ ]:
o7 = net[6](o6)
net[6]

In [ ]:
o7.shape

In [ ]:
o8 = net[7](o7)
net[7]

In [ ]:
o8.shape

In [ ]:
o9 = net[8](o8)
net[8]

In [ ]:
o9.shape

In [ ]:
o10 = net[9](o9)
net[9]

In [ ]:
o10.shape

# Data

PyTorch comes with a number of common datasets out of the box. The full list is available [here](https://pytorch.org/vision/stable/datasets.html).

In [ ]:
from torchvision import datasets, transforms
from tempfile import TemporaryDirectory

In [ ]:
d = TemporaryDirectory(prefix='dataset')
d.name

In [ ]:
tr_ds = datasets.CIFAR10(root=d.name, train=True, download=True, transform=transforms.ToTensor())
tr_ds

In [ ]:
val_ds = datasets.CIFAR10(root=d.name, train=False, download=True, transform=transforms.ToTensor())
val_ds

In [ ]:
# without transform
ds = datasets.CIFAR10(root=d.name, train=False, download=False)
ds[0]

In [ ]:
tr_dl  = DataLoader(tr_ds,  batch_size=4, shuffle=True,  num_workers=2)
val_dl = DataLoader(val_ds, batch_size=8, shuffle=False, num_workers=2)

In [ ]:
xb, yb = next(iter(tr_dl))
xb.shape

In [ ]:
show_img(xb[2].numpy().transpose(1,2,0));  # transpose required to go from CxHxW to HxWxC (PyTorch is channel-first)

## Standardisation

As before, we need to calculate per-channel pixel intensity mean ($\mu$) and standard deviation ($\sigma$) to be able to standardise/normalise our data. The problem is that for bigger datasets, you can't simply load it all to memory to calculate these two like we did with Fashion-MNIST. We will have to use a trick here instead!

The formulae for the mean and standard deviation that we have used before are:

$\mu ={\frac {1}{N}}\sum _{i=1}^{N}x_{i}$

$\sigma ={\sqrt {{\frac {1}{N}}\sum _{i=1}^{N}(x_{i}-\mu )^{2}}}$

The issue is that in order to calculate $\sigma$, we need to know $\mu$, so a naïve approach will require **two passess** over the dataset.

We can however take advantage of the following (see [Wikipedia](https://en.wikipedia.org/wiki/Standard_deviation#Definition_of_population_values)), where $\operatorname {E} [X]=\mu$:

${\displaystyle {\begin{aligned}\sigma &={\sqrt {\operatorname {E} [(X-\mu )^{2}]}}\\&={\sqrt {\operatorname {E} [X^{2}]+\operatorname {E} [-2\mu X]+\operatorname {E} [\mu ^{2}]}}\\&={\sqrt {\operatorname {E} [X^{2}]-2\mu \operatorname {E} [X]+\mu ^{2}}}\\&={\sqrt {\operatorname {E} [X^{2}]-2\mu ^{2}+\mu ^{2}}}\\&={\sqrt {\operatorname {E} [X^{2}]-\mu ^{2}}}\\&={\sqrt {\operatorname {E} [X^{2}]-(\operatorname {E} [X])^{2}}}\end{aligned}}}$



The above calculation in [Excel](https://livebournemouthac-my.sharepoint.com/:x:/g/personal/mbudka_bournemouth_ac_uk/EbEmpdAesrZHkuHg4ijDIwcBz2ILj_mcGOeEy8Z1rwqd1w?e=mZ7XBh).

In [ ]:
def get_stats(dl):
    cnt, csum, csum_sq = tc.zeros(1), tc.zeros(3), tc.zeros(3)  # accumulators; 3d because 3 colour channels

    for xb,_ in tqdm(dl):
        cnt += xb.shape[0]*xb.shape[2]*xb.shape[3]  # number of pixels per channel
        csum += xb.sum(dim=(0,2,3))                 # this gives 3 numbers i.e. sums of pixel intensities per each colour channel
        csum_sq += (xb**2).sum(dim=(0,2,3))         # ditto

    μ = csum / cnt
    σ = (csum_sq / cnt - μ**2).sqrt()
    return μ, σ

data_stats = get_stats(tr_dl)
data_stats

In [ ]:
transf = transforms.Compose([           # compose mutiple transforms; could use nn.Sequential() here instead of transforms.Compose()
    transforms.ToTensor(),
    transforms.Normalize(*data_stats)   # equivalent to transforms.Normalize(stl10_stats[0], stl10_stats[1])
])

In [ ]:
# tr_ds  = datasets.STL10(root=d.name, split='train', download=True, transform=transf)
# val_ds = datasets.STL10(root=d.name, split='test',  download=True, transform=transf)

tr_ds  = datasets.CIFAR10(root=d.name, train=True, download=True, transform=transf)
val_ds = datasets.CIFAR10(root=d.name, train=False,  download=True, transform=transf)

In [ ]:
bs = 8
tr_dl  = DataLoader(tr_ds,  batch_size=bs,   shuffle=True,  num_workers=2)
val_dl = DataLoader(val_ds, batch_size=2*bs, shuffle=False, num_workers=2)

In [ ]:
get_stats(tr_dl), get_stats(val_dl)  # sanity checks - we expect them to now be 0-mean, 1-std

In [ ]:
xb, yb = next(iter(tr_dl))
show_img(xb[1].numpy().transpose(1,2,0));
xb[0].min(), xb[0].max(), xb[0].mean(), xb[0].std()

# Training

## Network architecture

In [ ]:
# input: [bs,3,h,w] - this network will accept *any* input resolution
net = nn.Sequential(
    nn.Conv2d(in_channels=3,  out_channels=16, kernel_size=3, stride=1, padding=1), nn.LeakyReLU(), # [bs,16,h,w]
    nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3, stride=1, padding=1), nn.LeakyReLU(), # [bs,16,h,w]
    nn.MaxPool2d(kernel_size=2, stride=2),                                                          # [bs,16,h/2,w/2]
    nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1), nn.LeakyReLU(), # [bs,32,h/2,w/2]
    nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, stride=1, padding=1), nn.LeakyReLU(), # [bs,32,h/2,w/2]
    nn.MaxPool2d(kernel_size=2, stride=2),                                                          # [bs,32,h/4,w/4]
    # nn.Flatten(), nn.Linear(32*24*24, 10) # required for classification but this will tie the network to a fixed input resolution
).cuda()

In [ ]:
xb, yb = next(iter(tr_dl))
xb.shape

In [ ]:
o = net(xb.cuda())
o.shape

In [ ]:
net1 = nn.Sequential(
    *net[:-1],                   # use all the layers from 'net' (unpacking trick) without the last MaxPool2d
    nn.AdaptiveAvgPool2d((4,4))  # keras/tensorflow didn't have it until very recently!
)
net1

In [ ]:
net1(xb.cuda()).shape

In [ ]:
dummy = tc.randn((16, 3, 256, 256)).cuda()
net(dummy).shape, net1(dummy).shape

In [ ]:
dummy = tc.randn((16, 3, 128, 128)).cuda()
net(dummy).shape, net1(dummy).shape

In [ ]:
dummy = tc.randn((16, 3, 256, 128)).cuda()
net(dummy).shape, net1(dummy).shape

In [ ]:
# final network with classification head
net2 = nn.Sequential(
    *net[:-1],
    nn.AdaptiveAvgPool2d((4,4)),
    nn.Flatten(),
    nn.Linear(32*4*4,10)
).cuda()

In [ ]:
net2(dummy).shape

## Training loop

In [ ]:
from torch.amp import GradScaler, autocast

def fit(net, tr_dl, val_dl, loss=nn.CrossEntropyLoss(), epochs=3, lr=3e-3, wd=1e-3):

    Ltr_hist, Lval_hist = [], []

    scaler = GradScaler()  # for mixed-precision training

    opt = optim.AdamW(net.parameters(), lr=lr, weight_decay=wd)
    for epoch in trange(epochs):
        L = []
        dl = iter(tr_dl)
        for xb, yb in tqdm(dl, leave=False):
            xb, yb = xb.cuda(), yb.cuda()
            with autocast(device_type='cuda'):  # for mixed-precision training
                y_ = net(xb)
                l = loss(y_, yb)
            opt.zero_grad()
            scaler.scale(l).backward()  # previously l.backward()
            scaler.step(opt)            # previously opt.step()
            scaler.update()
            L.append(l.detach().cpu().numpy())

        Lval, Aval = [], []
        with tc.no_grad():
            for xb, yb in tqdm(iter(val_dl), leave=False):
                xb, yb = xb.cuda(), yb.cuda()
                with autocast(device_type='cuda'):
                    y_ = net(xb)
                    l = loss(y_, yb)
                Lval.append(l.detach().cpu().numpy())
                Aval.append((y_.max(dim=1)[1] == yb).float().mean().cpu().numpy())

        Ltr_hist.append(np.mean(L))
        Lval_hist.append(np.mean(Lval))
        print(f'training loss: {np.mean(L):0.4f}\tvalidation loss: {np.mean(Lval):0.4f}\tvalidation accuracy: {np.mean(Aval):0.2f}')
    return Ltr_hist, Lval_hist

In [ ]:
Ltr_hist, Lval_hist = fit(net2, tr_dl, val_dl, epochs=10)

In [ ]:
_,ax = plt.subplots(1,1,figsize=(20,4))
ax.plot(1+np.arange(len(Ltr_hist)),Ltr_hist)
ax.plot(1+np.arange(len(Lval_hist)),Lval_hist)
ax.grid('on')
ax.set_xlim(left=1, right=len(Ltr_hist))
ax.legend(['training loss', 'validation loss']);

# Homework

## For all

Train the neural network we have used in this notebook for maximum validation accuracy. Play with different values of the `learning rate` and `epochs`. Write down your results every time you train the network (i.e. for `lr=xx` and `epochs==yy`, `accuracy==zz`).

## For volunteers

Implement 2D convolution from scratch using loops.